In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:44
🔁 Restarting kernel...


In [ ]:
!conda install gh --channel conda-forge	

In [2]:
!pip install youtube-dl

     |████████████████████████████████| 1.9 MB 4.2 MB/s 


In [ ]:
!youtube-dl -i --write-sub --sub-lang ga -o '%(id)s.%(ext)s' PLbcLsUBW9b3BvuTbtMKBXmuyJq6Ygg3Rg

[download] Destination: F1PZYl4Wcx0.f137.mp4
[download]  74.2% of 52.81MiB at  2.80MiB/s ETA 00:04

In [ ]:
!ls ./*.vtt|zip subtitles.zip -@

In [6]:
from google.colab import files
files.download('subtitles.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>